In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load the trained model
model = load_model('chatbot_model.h5')

# Load intents file
intents = json.loads(open('intents.json').read())

# Load words and classes from pickle files
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [6]:
def clean_up_sentence(sentence):
    # Tokenize the pattern - split words into an array
    sentence_words = nltk.word_tokenize(sentence)
    
    # Lemmatize each word - create the base form for each word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    """
    Return a bag of words array: 0 or 1 for each word in the bag 
    that exists in the sentence.
    """
    # Tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # Bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # Assign 1 if the current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    """
    Predict the class of the given sentence using the trained model.
    """
    # Filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    # Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [7]:
def getResponse(ints, intents_json):
    """
    Retrieve a response based on the predicted intent.
    
    Parameters:
    - ints: List of intents with probabilities returned by `predict_class`.
    - intents_json: JSON object containing all intents and their responses.
    
    Returns:
    - A randomly selected response corresponding to the predicted intent.
    """
    tag = ints[0]['intent']  # Get the top predicted intent
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if i['tag'] == tag:  # Match the intent tag
            result = random.choice(i['responses'])  # Select a random response
            break
    return result

def chatbot_response(text):
    """
    Generate a chatbot response for the given input text.
    
    Parameters:
    - text: User input string.
    
    Returns:
    - A string response from the chatbot.
    """
    ints = predict_class(text, model)  # Predict the intent
    res = getResponse(ints, intents)  # Get the corresponding response
    return res

In [8]:
import tkinter
from tkinter import *

def send():
    """
    Handles the sending of a message from the user and displaying the bot's response.
    """
    # Get user input from EntryBox
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)
    
    if msg != '':
        # Enable ChatLog for editing
        ChatLog.config(state=NORMAL)
        
        # Insert user's message into the chat window
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))
        
        # Get bot response and insert it into the chat window
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        
        # Disable ChatLog to prevent editing and scroll to the end
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

# Initialize the main Tkinter window
base = Tk()
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

# Create ChatLog - a text widget to display conversation
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

# Create a scrollbar and bind it to ChatLog
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Create Send Button
SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff', command=send)

# Create EntryBox - a text widget for user to type messages
EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial")
# Uncomment the line below to enable sending messages with the Enter key
# EntryBox.bind("<Return>", send)

# Place all components on the window
scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

# Start the Tkinter event loop
base.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\NIFFELHEIM\AppData\Local\Temp\ipykernel_936\4172229907.py", line 21, in send
    res = chatbot_response(msg)
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NIFFELHEIM\AppData\Local\Temp\ipykernel_936\3101043445.py", line 31, in chatbot_response
    ints = predict_class(text, model)  # Predict the intent
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NIFFELHEIM\AppData\Local\Temp\ipykernel_936\1212367674.py", line 33, in predict_class
    p = bow(sentence, words, show_details=False)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NIFFELHEIM\AppData\Local\Temp\ipykernel_936\1212367674.py", line 15, in bow
    sentence_words = clean_up_sentence(sentence)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\NIFFELHEIM\AppData\Local\Temp\ipykerne